# 1. 크롤링

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
import time
import pandas as pd

In [2]:
import pickle

In [18]:
service = Service('chromedriver.exe')
driver = webdriver.Chrome(service = service)

In [4]:
isbn13 = pd.read_pickle('isbn13.pkl')

In [6]:
isbn13

0        9791190710343
1        9791190710350
2        9791190710367
3        9791128858338
4        9788954685603
             ...      
54971    9791185020136
54972    9788997838127
54973    9788956591858
54974    9788987794938
54975    9788995696972
Name: isbn13, Length: 54976, dtype: object

In [ ]:
rank_mean = [] # 도서 평균 평점
rank_percentage = [] # 평균 percentage
review_title = [] # 리뷰 제목
review_id = [] # 리뷰 id
review_rank = [] # 리뷰 평점
review_text = [] # 리뷰 
isbn_idx = []


for isbn in isbn13[3041:4000]: # 페이지 입력
    url = "http://www.yes24.com/product/search?query={}".format(isbn)
    driver.get(url)
    time.sleep(2)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    # 페이지에 있는 도서 링크(50개)
    book_links = soup.select('a.gd_name')
    goods = []
    for link in book_links:
        a= link['href'].split('/')[3] 
        goods.append(a)
    for item in goods:
        del(goods[1:])
        driver.get('http://www.yes24.com'+'/Product/Goods/{}'.format(item)) # 상품 페이지
        time.sleep(1)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        try:
            mean = soup.select('div.gd_rating > em.yes_b')[0].text
            rank_mean.append([mean]) # 도서 평균 평점
        except:
            mean = []
            rank_mean.append(mean) # 도서 평균 평점
        rank_p = []
        try:
            for k in range(5):
                rank_ = soup.select('span.barGraphBase')[k].text # 평점 percentage
                rank_p.append(rank_)
            rank_percentage.append(rank_p)
        except:
            rank_ = []
            rank_percentage.append(rank_)
        try:
            review_t = []  # 각각의 책의 리뷰제목 리스트
            review_i = []
            review_r = []
            review_te = []
            for page in range(1,3): # 리뷰 페이지 2개
                driver.get('http://www.yes24.com/Product/communityModules/GoodsReviewList/{}?Type=ALL&_=1648001707174&Sort=2&PageNumber={}'.format(item, page))
                time.sleep(1)
                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')
                
                blog_link = []
                for i in range(5): # 리뷰 한 페이지에 있는 5개 리뷰 가져오기
                    try:
                        title = soup.select('span.review_tit')[i].text.strip() # 리뷰 제목
                        review_t.append(title)
                        r_id = soup.select('a.lnk_id')[i].text # 리뷰 id
                        review_i.append(r_id)
                        rank = soup.select('span.review_rating')[i].text.split('\n')[1][-2:] # 리뷰 평점
                        review_r.append(rank)
                        blog_l = soup.select('div.review_lnk a')[i]['href'] # 리뷰 블로그 링크
                        blog_link.append(blog_l)
                    except:
                        pass
                for j in range(5): # 리뷰 한 페이지에 있는 5개 블로그 내용 가져오기
                        driver.get(blog_link[j]) # 리뷰 블로그 링크 들어가기
                        time.sleep(1)
                        html = driver.page_source
                        soup = BeautifulSoup(html, 'html.parser')
                        text = soup.select('div.blogContArea')[0].text # 리뷰 내용 
                        review_te.append(text)
        except:
            pass
        review_title.append(review_t)
        review_id.append(review_i)
        review_rank.append(review_r)
        review_text.append(review_te)
        isbn_idx.append(isbn)

In [20]:
col = ['isbn','rank_mean','rank_percentage','review_title','review_id','review_rank','review_text']
user = pd.DataFrame([isbn_idx,rank_mean,rank_percentage,review_title,review_id,review_rank,review_text]).T
user.columns = col
user

,rank_mean,rank_percentage,review_title,review_id,review_rank,review_text
0,[8.8],"[80%, 0%, 20%, 0%, 0%]","[2022-05 우리를 아끼기로 합니다, 우리를 아끼기로 합니다, 우리를 아끼기로 ...","[져*, k*******2, n********1, y******y, d*****9]","[5점, 5점, 5점, 4점, 3점]",[\np.25\n그래서 종종 펜을 든다. 편지지가 아니더라도 쓴다. 편지를 쓰는 일...
1,[9.0],"[50%, 50%, 0%, 0%, 0%]","[꿋꿋한 그녀들의 삶, 나는 파독 간호사입니다, 나는 파독간호사입니다]","[키*스, 목***수]","[4점, 5점]",[\n영화 <국제시장>중 내 눈길을 사로잡았던 건 아무래도 흥남부두에서 철수하는 장...
2,[10.0],"[100%, 0%, 0%, 0%, 0%]","[빌리 아일리시, 빌리 아일리시, 빌리 아일리시, 빌리 아일리시 04, (빌리 아일...","[N****e, 정*지, c*******3, c*******3, 로*나, 융*이, ...","[5점, 5점, 5점, 5점, 5점, 5점, 5점, 5점, 5점, 5점]",[\n내가 빌리 아일리시를 처음 알게 된 건 아이러니하게도 그의 음악을 통해서가 아...
3,[10.0],"[100%, 0%, 0%, 0%, 0%]","[가끔 요리도 합니다. 요리 에세이(딜레르), 가끔 요리도 합니다, 가끔 요리도 합...","[러***니, 류*, p******7, t*******4, n******n]","[5점, 5점, 5점, 5점, 5점]","[\n그림 그리는 보노의 나를 위한 요리요리 에세이""가끔 요리도 합니다.""왠지 요리..."


In [10]:
user.to_pickle('user_3041-3040', protocol = 4)

In [15]:
user1 = pd.read_pickle('user_3000-3040.pkl') # 수정 전